# Prepare  loan DB & add location

In [1]:
import pandas as pd

# Load loan file

df=pd.read_csv("data/cr_loan2.csv")
df.reset_index(inplace=True)
print(df.columns)

Index(['index', 'person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_status', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length'],
      dtype='object')


In [2]:
# Clean up load file

indices=df[df['loan_status'].isnull()].index
df.drop(indices,inplace=True)

indices=df[df['person_emp_length'].isnull()].index
df.drop(indices,inplace=True)

df.dropna(inplace=True)

df['loan_percent_income']=df['loan_percent_income'].astype('float')
df['loan_amnt']=df['loan_amnt'].astype('float')
df['person_income']=df['person_income'].astype('float')
df['loan_int_rate']=df['loan_int_rate'].astype('float')
df['loan_intent']=df['loan_intent'].astype(str)
df['loan_grade']=df['loan_grade'].astype(str)

filter=(df['person_emp_length']<40) & (df['person_income']<500000)
df=df[filter]


df['age_type']=df['person_age'].apply(lambda row: "teenager" if (row<20)
                                     else "young_adult" if (row>=20) & (row<30)
                                    else "adult" if (row>=30) & (row < 50)
                                      else "senior"
                                     )

#encoder=LabelEncoder()
#df["person_home_ownership_enc"]=encoder.fit_transform(df["person_home_ownership"])
#df["loan_grade_enc"]=encoder.fit_transform(df["loan_grade"])
#df["cb_person_default_on_file_enc"]=encoder.fit_transform(df["cb_person_default_on_file"])

#rint(df.info())
cred_num  =df.select_dtypes(exclude=['object'])
cred_cat = df.select_dtypes(include=['object'])
ohe=pd.get_dummies(cred_cat)
df=pd.concat([df,ohe],axis=1)

print(df.columns)


Index(['index', 'person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_status', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length', 'age_type',
       'person_home_ownership_MORTGAGE', 'person_home_ownership_OTHER',
       'person_home_ownership_OWN', 'person_home_ownership_RENT',
       'loan_intent_DEBTCONSOLIDATION', 'loan_intent_EDUCATION',
       'loan_intent_HOMEIMPROVEMENT', 'loan_intent_MEDICAL',
       'loan_intent_PERSONAL', 'loan_intent_VENTURE', 'loan_grade_A',
       'loan_grade_B', 'loan_grade_C', 'loan_grade_D', 'loan_grade_E',
       'loan_grade_F', 'loan_grade_G', 'cb_person_default_on_file_N',
       'cb_person_default_on_file_Y', 'age_type_adult', 'age_type_senior',
       'age_type_young_adult'],
      dtype='object')


In [3]:
df

,index,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,...,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_N,cb_person_default_on_file_Y,age_type_adult,age_type_senior,age_type_young_adult
1,1,21,9600.0,OWN,5.0,EDUCATION,B,1000.0,11.14,0,...,0,0,0,0,0,1,0,0,0,1
2,2,25,9600.0,MORTGAGE,1.0,MEDICAL,C,5500.0,12.87,1,...,1,0,0,0,0,1,0,0,0,1
3,3,23,65500.0,RENT,4.0,MEDICAL,C,35000.0,15.23,1,...,1,0,0,0,0,1,0,0,0,1
4,4,24,54400.0,RENT,8.0,MEDICAL,C,35000.0,14.27,1,...,1,0,0,0,0,0,1,0,0,1
5,5,21,9900.0,OWN,2.0,VENTURE,A,2500.0,7.14,1,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32576,32576,57,53000.0,MORTGAGE,1.0,PERSONAL,C,5800.0,13.16,0,...,1,0,0,0,0,1,0,0,1,0
32577,32577,54,120000.0,MORTGAGE,4.0,PERSONAL,A,17625.0,7.49,0,...,0,0,0,0,0,1,0,0,1,0
32578,32578,65,76000.0,RENT,3.0,HOMEIMPROVEMENT,B,35000.0,10.99,1,...,0,0,0,0,0,1,0,0,1,0
32579,32579,56,150000.0,MORTGAGE,5.0,PERSONAL,B,15000.0,11.48,0,...,0,0,0,0,0,1,0,0,1,0


In [4]:
# Add location city to loan df
city_df=pd.read_csv("data/european_cities.csv")
city_df

,country,city
0,GERMANY,ALTSTADT
1,ITALY,LENNA
2,ITALY,MASCALI
3,FRANCE,LANTOSQUE
4,FRANCE,VALLAURIS
...,...,...
586,TURKEY,PATNOS
587,ITALY,CORDENONS
588,GERMANY,EBNATH
589,ITALY,VALLECROSIA


In [5]:
import numpy as np

data_rd = np.random.randint(0,len(city_df)-1,size=len(df) )

In [11]:
max(data_rd)

589

In [7]:
city_rd = city_df.iloc[data_rd]

In [13]:
city_rd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28586 entries, 343 to 193
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  28586 non-null  object
 1   city     28586 non-null  object
dtypes: object(2)
memory usage: 670.0+ KB


In [18]:
city_rd.index= pd.RangeIndex(start=0, stop=len(df), step=1)
df.index= pd.RangeIndex(start=0, stop=len(df), step=1)

In [19]:
city_rd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28586 entries, 0 to 28585
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  28586 non-null  object
 1   city     28586 non-null  object
dtypes: object(2)
memory usage: 446.8+ KB


In [21]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28586 entries, 0 to 28585
Data columns (total 37 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   index                           28586 non-null  int64  
 1   person_age                      28586 non-null  int64  
 2   person_income                   28586 non-null  float64
 3   person_home_ownership           28586 non-null  object 
 4   person_emp_length               28586 non-null  float64
 5   loan_intent                     28586 non-null  object 
 6   loan_grade                      28586 non-null  object 
 7   loan_amnt                       28586 non-null  float64
 8   loan_int_rate                   28586 non-null  float64
 9   loan_status                     28586 non-null  int64  
 10  loan_percent_income             28586 non-null  float64
 11  cb_person_default_on_file       28586 non-null  object 
 12  cb_person_cred_hist_length      

In [22]:
df[['country','city']] = city_rd[['country','city']]
#c=city_df.iloc[data_rd].reindex()
#df.columns

In [23]:
df

,index,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,...,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_N,cb_person_default_on_file_Y,age_type_adult,age_type_senior,age_type_young_adult,country,city
0,1,21,9600.0,OWN,5.0,EDUCATION,B,1000.0,11.14,0,...,0,0,0,1,0,0,0,1,ITALY,GIARRE
1,2,25,9600.0,MORTGAGE,1.0,MEDICAL,C,5500.0,12.87,1,...,0,0,0,1,0,0,0,1,TURKEY,PAZAR
2,3,23,65500.0,RENT,4.0,MEDICAL,C,35000.0,15.23,1,...,0,0,0,1,0,0,0,1,ITALY,VILLAPUTZU
3,4,24,54400.0,RENT,8.0,MEDICAL,C,35000.0,14.27,1,...,0,0,0,0,1,0,0,1,ITALY,CATANIA
4,5,21,9900.0,OWN,2.0,VENTURE,A,2500.0,7.14,1,...,0,0,0,1,0,0,0,1,FRANCE,MONTELIMAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28581,32576,57,53000.0,MORTGAGE,1.0,PERSONAL,C,5800.0,13.16,0,...,0,0,0,1,0,0,1,0,UNITED KINGDOM,EDEN
28582,32577,54,120000.0,MORTGAGE,4.0,PERSONAL,A,17625.0,7.49,0,...,0,0,0,1,0,0,1,0,FRANCE,VILLEGAILHENC
28583,32578,65,76000.0,RENT,3.0,HOMEIMPROVEMENT,B,35000.0,10.99,1,...,0,0,0,1,0,0,1,0,ITALY,CLAVESANA
28584,32579,56,150000.0,MORTGAGE,5.0,PERSONAL,B,15000.0,11.48,0,...,0,0,0,1,0,0,1,0,TURKEY,ILGIN


In [24]:
import cml.data_v1 as cmldata

CONNECTION_NAME = "go01-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()




Setting spark.hadoop.yarn.resourcemanager.principal to oliviermeignan


In [ ]:
# spark.sparkContext._conf.getAll()

In [ ]:
# Sample usage to run query through spark
db="creditrisk"
spark.sql("create database if not exists " + db)

sdf=spark.createDataFrame(df)

sdf.write.format("parquet").mode("overwrite").saveAsTable(db + ".loan")

#sdf.writeTo(db + ".loan").using("parquet").create()

#sdf.writeTo("spark_catalog." + db + ".loan").
spark.sql("SHOW CURRENT NAMESPACE").show()
#spark.sql("select * from default.b").show()

#.config("spark.sql.catalog.local","org.apache.iceberg.spark.SparkCatalog")


In [ ]:
spark.stop()